In [1]:
from datasets import load_dataset

# Load the TinyStories dataset
dataset = load_dataset("roneneldan/TinyStories")

# Print dataset information
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})


In [2]:
# Function to sample 5% of the dataset with optional seed and offset
def sample_five_percent(dataset_split, seed=42, offset=0):
    total_size = len(dataset_split)
    five_percent_size = total_size // 20  # 5% of the dataset
    shuffled = dataset_split.shuffle(seed=seed)
    return shuffled.select(range(offset, offset + five_percent_size))

# Sample 5% from train, validation, and test splits
train_data = sample_five_percent(dataset['train'], seed=42)
val_data   = sample_five_percent(dataset['validation'], seed=123, offset=0)
test_data  = sample_five_percent(dataset['validation'], seed=123, offset=len(val_data))  # next 5%

# Check sizes
print(f"Train size (5%): {len(train_data)}")
print(f"Validation size (5%): {len(val_data)}")
print(f"Test size (5%): {len(test_data)}")

# Show first five samples from val_data
print("\n=== First 5 validation samples ===")
for i in range(5):
    print(f"{i + 1}: {val_data[i]}")

# Show first five samples from test_data
print("\n=== First 5 test samples ===")
for i in range(5):
    print(f"{i + 1}: {test_data[i]}")


Train size (5%): 105985
Validation size (5%): 1099
Test size (5%): 1099

=== First 5 validation samples ===
1: {'text': 'Once upon a time, there was a princess who had an awful problem. She hated her weight. Everywhere she went she felt so sorry and sad.\n\nThe princess wanted so badly to be thin like the other animals in her kingdom. She tried to do everything to make her weight go away - she ate less and ran and jumped, but nothing worked.\n\nOne day, something magical happened. A magical fairy appeared and waved her wand and said, "No more weight for the princess". And, just like that, the princess was free from her weight.\n\nThe princess was so, so happy. She thanked the fairy again and again and ran off dancing and singing.\n\nThe princess never worried about her weight again, she was so happy, and thanked the magical fairy every day.'}
2: {'text': "Once upon a time, there was a little boy named Timmy. Timmy was very hungry and wanted to eat a cookie. But his mom said he had to e

In [4]:
from transformers import GPT2Tokenizer

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set padding token to EOS token to avoid warnings
tokenizer.pad_token = tokenizer.eos_token

In [5]:
# Function to tokenize and prepare inputs/labels
def tokenize_function_with_labels(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        padding="max_length",  # Pad to a fixed length for batches
        max_length=512         # Set maximum length for sequences
    )
    # Add labels (same as input_ids for language modeling)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Tokenize train and validation datasets
train_dataset = train_data.map(tokenize_function_with_labels, batched=True, remove_columns=["text"])
val_dataset = val_data.map(tokenize_function_with_labels, batched=True, remove_columns=["text"])

# Verify tokenized data structure
print(train_dataset[0])

{'input_ids': [14967, 290, 32189, 588, 284, 711, 287, 262, 3952, 13, 1119, 766, 257, 1263, 3430, 319, 262, 2323, 13, 632, 318, 7586, 290, 890, 290, 4334, 13, 198, 198, 1, 8567, 11, 257, 3430, 2474, 5045, 1139, 13, 366, 40, 460, 10303, 340, 2474, 198, 198, 1544, 8404, 284, 10303, 262, 3430, 11, 475, 340, 318, 1165, 5802, 13, 679, 8953, 866, 290, 10532, 262, 3430, 13, 198, 198, 1, 46, 794, 2474, 339, 1139, 13, 366, 2504, 5938, 2474, 198, 198, 44, 544, 22051, 13, 1375, 318, 407, 1612, 11, 673, 655, 6834, 340, 318, 8258, 13, 198, 198, 1, 5756, 502, 1949, 2474, 673, 1139, 13, 366, 40, 460, 5236, 340, 2474, 198, 198, 3347, 11103, 510, 262, 3430, 290, 7584, 340, 319, 607, 1182, 13, 1375, 11114, 6364, 290, 7773, 13, 1375, 857, 407, 2121, 866, 13, 198, 198, 1, 22017, 2474, 5045, 1139, 13, 366, 1639, 389, 922, 379, 22486, 2474, 198, 198, 1, 10449, 345, 2474, 32189, 1139, 13, 366, 1026, 318, 1257, 2474, 198, 198, 2990, 1011, 4962, 22486, 262, 3430, 319, 511, 6665, 11, 5101, 11, 290, 7405, 13, 111

In [6]:
from transformers import GPT2LMHeadModel

# Load GPT-2 model with a language modeling head
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Resize embeddings to account for padding token
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [7]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-tinystories3",  # Directory to save model checkpoints
    evaluation_strategy="steps",     # Evaluate at specific intervals
    eval_steps=500,                  # Evaluate every 500 steps
    learning_rate=5e-5,              # Learning rate for optimizer
    weight_decay=0.01,               # Regularization strength
    per_device_train_batch_size=4,   # Training batch size
    per_device_eval_batch_size=4,    # Evaluation batch size
    num_train_epochs=3,              # Total number of training epochs
    save_strategy="steps",           # Save model at specific intervals
    save_steps=500,                  # Save model every 500 steps
    logging_dir="./logs",            # Directory for training logs
    save_total_limit=3,              # Limit total number of saved checkpoints
    load_best_model_at_end=True,     # Load best model after training
    save_safetensors=False
)

C:\Users\ksedr\AppData\Roaming\Python\Python311\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
pip install tf-keras


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------------------------ --------- 1.3/1.7 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 4.5 MB/s eta 0:00:00
Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.3.77 requires numpy<=2.1.1,>=1.23.0, but you have numpy 2.1.3 which is incompatible.
tensorflow-intel 2.13.1 requires keras<2.14,>=2.13.1, but you have keras 3.9.0 which is incompatible.
tensorflow-intel 2.13.1 requires numpy<=1.24.3,>=1.22, but you have numpy 2.1.3 which is incompatible.
tensorflow-intel 2.13.1 requires tensorboard<2.14,>=2.13, but you have tensorboard 2.19.0 which is incompatible.
tensorflow-intel 2.13.1 requires tensorflow-estimator<2.14,>=2.13.0, but you have tensorflow-estimator 2.12.0 which is incompatible.
tensorflow-intel 2.13.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing

In [14]:
from transformers import EarlyStoppingCallback

# Add EarlyStoppingCallback to stop training if validation loss does not improve
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)

In [8]:
from transformers import Trainer

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    
)

C:\Users\ksedr\AppData\Local\Temp\ipykernel_33080\1833781753.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
# Train the model
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
500,0.883700,0.768094
1000,0.799800,0.740806
1500,0.769700,0.723584
2000,0.764600,0.709567
2500,0.745900,0.699948
3000,0.733100,0.692197
3500,0.732400,0.686280
4000,0.728400,0.680139
4500,0.722500,0.675569
5000,0.719900,0.671114


TrainOutput(global_step=79491, training_loss=0.6278838022936559, metrics={'train_runtime': 37723.3402, 'train_samples_per_second': 8.429, 'train_steps_per_second': 2.107, 'total_flos': 8.307910803456e+16, 'train_loss': 0.6278838022936559, 'epoch': 3.0})

In [10]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "Once upon a time in a magical forest"
generated_story = story_generator(prompt, max_length=150, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated Story:
Once upon a time in a magical forest, there lived a brave knight. Everyday he would wear his special helmet. Everyday he would set off on an adventure. He would explore the forest and discover new things.

One day the knight heard a strange sound. It was thundering and it made him very frightened. He was about to go home but his helmet saved him! He slowly put it on and before he knew it, he was safe.

The knight decided that it was time to set and to go home without any worries. He ran back and forth but nothing happened but he knew that he had been brave. 

Finally, when the knight was safe by a tree, he thanked God and promised to always keep his


In [11]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "One day I was walking in an ancient desert when"
generated_story = story_generator(prompt, max_length=150, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])

Device set to use cuda:0


Generated Story:
One day I was walking in an ancient desert when dark clouds started to pour down. It was pouring rain, and it was getting cold. Suddenly a thunderstorm came and it rained. It's power was out and the field was so empty. But then a tall figure came out of the clouds. It was a giant elephant!

The elephant looked and he said 'What are you doing?'
The poor elephant looked down and said 'I'm pouring electricity!'
The elephant looked out of the window and said 'Oh! That's why I look out to the sky, I'm able to help protect plants and plants from their worst fears.'

The elephant smiled. With little help and thoughtfulness, he had saved the day


In [12]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "I know a lion with a weird big nose"
generated_story = story_generator(prompt, max_length=250, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])

Device set to use cuda:0


Generated Story:
I know a lion with a weird big nose. He likes to dance and sing! He dances in the dirt and the grass.

One day, a little boy sees the lion. He loves lions and he wants to dance with him.

The lion says, "Come, little boy! dance with me here!" The little boy is excited and says, "Yes! Let's dance!"

So, the lion and the little boy go to the dirt and dance together. They dance loud and fast with their little nose! It makes them feel happy and funny.

Soon, they are fast asleep and the lion and the little boy dance. They dance together in the dirt and the grass until the sun goes down.

The little boy and the lion never catch another lion again. They dance happily together in the middle of the night.


In [13]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "I know a giraffe with a little neck"
generated_story = story_generator(prompt, max_length=250, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])

Device set to use cuda:0


Generated Story:
I know a giraffe with a little neck. He liked to play in the sunshine. One day, the hill started to shake, and the giraffe felt very frightened. The other animals ran away from him.

"Help us, giraffe!" one duck said. "We must find a way past it."

The other animals looked for a way, but they couldn't find one. The giraffe felt very sad and lonely. He started to yawn. 

The giraffe slowly walked back to his forest. As he walked, he noticed that some of his friends were trying to cross the bridge. They were shaking over and over again. The giraffe was not scared anymore. 

The giraffe stopped yawning and thought to himself, "I can handle this alone. Maybe one day, another way will help us."


In [14]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "I know a leon with a cute tail"
generated_story = story_generator(prompt, max_length=250, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])

Device set to use cuda:0


Generated Story:
I know a leon with a cute tail. It was a rare leon because it always saw the sun shining on it. It was very happy with its tail and flew around in the sky.

One day, the leon saw something bright on the ground. It saw a tiny bird. The leon thought the bird was nice, so it flew closer to it. The bird cooed with it and the leon saw that it was trying to learn.

The leon felt happy that the bird liked it. The leon was happy too because it had a good friend and their love. They played together, and the leon taught the bird to like it and share its love with everyone.


In [17]:
# Evaluate the model on the validation set
print("Evaluating model on validation set...")

eval_results = trainer.evaluate()

print("=== VALIDATION SET EVALUATION RESULTS ===")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")




Evaluating model on validation set...


=== VALIDATION SET EVALUATION RESULTS ===
eval_loss: 0.5725
eval_runtime: 40.0149
eval_samples_per_second: 27.4650
eval_steps_per_second: 6.8720
epoch: 3.0000


NameError: name 'np' is not defined

In [18]:
import numpy as np

# Calculate perplexity
perplexity = np.exp(eval_results["eval_loss"])
print(f"\nPerplexity: {perplexity:.2f}")

# Interpret model quality based on perplexity
if perplexity < 50:
    quality = "Excellent"
elif perplexity < 100:
    quality = "Good"
elif perplexity < 200:
    quality = "Fair"
else:
    quality = "Needs Improvement"

print(f"Model Quality: {quality}")


Perplexity: 1.77
Model Quality: Excellent


In [ ]:
from transformers import pipeline
from collections import Counter
import numpy as np

def compute_distinct_n(texts, n=1):
    ngrams = []
    for text in texts:
        tokens = text.strip().split()
        ngrams.extend([tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)])
    total = len(ngrams)
    unique = len(set(ngrams))
    return unique / total if total > 0 else 0.0


In [6]:
import os
import numpy as np
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


In [5]:
!pip install sentence_transformers

  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached scikit_learn-1.7.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (17 kB)
  Using cached scipy-1.16.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (61 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached sentence_transformers-5.0.0-py3-none-any.whl (470 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 5.5 MB/s eta 0:00:000:00:01m eta 0:00:01
Using cached scikit_learn-1.7.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.5 MB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Using cached scipy-1.16.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (35.1 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [sentence_transformers]/6 [sentence_transformers]


In [7]:
# Update model_path with your checkpoint directory
model_path = "/home/kiro/Downloads/Project/gpt2-tinystories3/checkpoint-79491"

dataset_name       = "roneneldan/TinyStories"
split_name         = "validation"
prompt_token_count = 20    # number of tokens to use as prompt
num_generations    = 5     # number of stories per prompt for diversity
max_gen_length     = 100   # max total tokens (prompt + generation)
num_examples       = 300   # set None to evaluate all examples


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

tokenizer = GPT2Tokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
model.eval()

embedder = SentenceTransformer('all-MiniLM-L6-v2', device=device)


Using device: cuda


In [9]:
ds = load_dataset(dataset_name, split=split_name)


Generating train split: 100%|█| 2119719/2119719 [00:03<00:00, 699680.85 examples
Generating validation split: 100%|█| 21990/21990 [00:00<00:00, 707979.56 example


In [10]:
cosine_scores  = []
overlap_scores = []
diversity_scores = []


In [13]:
for idx, example in enumerate(ds):
    text = example["text"].strip()
    tokens = tokenizer.tokenize(text)

    if len(tokens) <= prompt_token_count + 1:
        continue

    prompt_tokens = tokens[:prompt_token_count]
    gold_tokens   = tokens[prompt_token_count:]
    prompt = tokenizer.convert_tokens_to_string(prompt_tokens)
    gold   = tokenizer.convert_tokens_to_string(gold_tokens)

    # Tokenize prompt and move to GPU
    encoded = tokenizer(prompt, return_tensors="pt", padding=True)
    input_ids = encoded.input_ids.to(device)
    attention_mask = encoded.attention_mask.to(device)

    # Generate sequences
    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=min(len(input_ids[0]) + len(gold_tokens), max_gen_length),
        do_sample=True,
        top_k=50,
        top_p=0.95,
        num_return_sequences=num_generations,
        pad_token_id=tokenizer.pad_token_id
    )

    # Decode generated texts
    gens = []
    for gen in output_ids:
        gen_tok = gen[input_ids.shape[1]:].tolist()
        gens.append(tokenizer.decode(gen_tok, skip_special_tokens=True).strip())

    # Cosine similarity
    emb_prompt = embedder.encode([prompt])
    emb_gold   = embedder.encode([gold])
    cs = float(cosine_similarity(emb_prompt, emb_gold)[0, 0])
    cosine_scores.append(cs)

    # Unigram overlap
    set_prompt = set(prompt_tokens)
    set_gold   = set(tokenizer.tokenize(gold))
    overlap = len(set_prompt & set_gold) / len(set_prompt)
    overlap_scores.append(overlap)

    # Diversity (distinct-1)
    all_gen_toks = []
    for g in gens:
        all_gen_toks.extend(tokenizer.tokenize(g))
    diversity = len(set(all_gen_toks)) / len(all_gen_toks) if all_gen_toks else 0.0
    diversity_scores.append(diversity)

    # 🔄 Progress update
    print(f"[{len(cosine_scores)}/{num_examples or '∞'}] ✅ "
          f"Len(prompt): {len(prompt_tokens)} | Len(gold): {len(gold_tokens)} | "
          f"Cosine: {cs:.4f} | Overlap: {overlap:.4f} | Diversity: {diversity:.4f}")

    if num_examples and len(cosine_scores) >= num_examples:
        print("✅ Evaluation complete.")
        break


[129/300] ✅ Len(prompt): 20 | Len(gold): 62 | Cosine: 0.5213 | Overlap: 0.4706 | Diversity: 0.3689
[130/300] ✅ Len(prompt): 20 | Len(gold): 283 | Cosine: 0.2717 | Overlap: 0.4118 | Diversity: 0.3600
[131/300] ✅ Len(prompt): 20 | Len(gold): 111 | Cosine: 0.5228 | Overlap: 0.5714 | Diversity: 0.3267
[132/300] ✅ Len(prompt): 20 | Len(gold): 171 | Cosine: 0.5954 | Overlap: 0.5556 | Diversity: 0.3475
[133/300] ✅ Len(prompt): 20 | Len(gold): 124 | Cosine: 0.6327 | Overlap: 0.6000 | Diversity: 0.3609
[134/300] ✅ Len(prompt): 20 | Len(gold): 181 | Cosine: 0.4135 | Overlap: 0.4706 | Diversity: 0.3775
[135/300] ✅ Len(prompt): 20 | Len(gold): 133 | Cosine: 0.4198 | Overlap: 0.5789 | Diversity: 0.3625
[136/300] ✅ Len(prompt): 20 | Len(gold): 131 | Cosine: 0.3530 | Overlap: 0.5789 | Diversity: 0.3648
[137/300] ✅ Len(prompt): 20 | Len(gold): 132 | Cosine: 0.5729 | Overlap: 0.5294 | Diversity: 0.3951
[138/300] ✅ Len(prompt): 20 | Len(gold): 205 | Cosine: 0.6095 | Overlap: 0.6250 | Diversity: 0.3516
[

In [14]:
def summarize(name, arr):
    arr = np.array(arr)
    return f"{name}: mean={arr.mean():.4f}, std={arr.std():.4f}"

print("\n=== Evaluation Results ===")
print(summarize("Cosine similarity (prompt vs gold)", cosine_scores))
print(summarize("Unigram overlap  (prompt vs gold)", overlap_scores))
print(summarize("Diversity (distinct-1)", diversity_scores))



=== Evaluation Results ===
Cosine similarity (prompt vs gold): mean=0.5066, std=0.0914
Unigram overlap  (prompt vs gold): mean=0.5249, std=0.0916
Diversity (distinct-1): mean=0.3763, std=0.0278


In [15]:
# Replace with any custom prompt
your_prompt = "Once upon a time, a little robot wanted to learn"

# Tokenize and move to GPU
encoded = tokenizer(your_prompt, return_tensors="pt", padding=True)
input_ids = encoded.input_ids.to(device)
attention_mask = encoded.attention_mask.to(device)

# Generate sequences
output_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_length=100,                # total tokens including prompt
    do_sample=True,
    top_k=50,
    top_p=0.95,
    num_return_sequences=5,        # generate 5 completions
    pad_token_id=tokenizer.pad_token_id
)

# Decode and show generations
print("=== Generated Completions ===\n")
for i, gen in enumerate(output_ids):
    generated = tokenizer.decode(gen[input_ids.shape[1]:], skip_special_tokens=True)
    print(f"[{i+1}] {generated.strip()}\n")


=== Generated Completions ===

[1] how to fly. He asked his mom, "Can you teach me how to fly?" His mom said, "Sure, let's practice together." 

As they practiced, the robot started to fly. He started to climb trees and swing on the branches. He flew so high that he felt like he was flying!

He tried to stay focused so he could fly. He even got to the other side of a big hill

[2] how to play. He asked his friends all over the world if he could join them, but he still couldn't figure out how. He wanted to start and he was worried.

But one day he got an idea. He asked his friends in the garden to try and teach him. His friends were happy to help and together they began to teach the little robot how to play.

The little robot was amazed by what they were

[3] how to cook. So, it asked its friend, a wise owl, if it could help. The owl said yes, and the robot learned how to make it very happy.

But, the robot was not happy with the robot. The robot wanted to keep trying, but it realized i

In [17]:
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import torch
from tqdm import tqdm

# Load model from final checkpoint
checkpoint_path = "/home/kiro/Downloads/Project/gpt2-tinystories3/checkpoint-79491"  # adjust to actual final step
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint_path)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(checkpoint_path).to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

# Sample same 5% val split
val_full = load_dataset("roneneldan/TinyStories", split="validation")
val_data = val_full.shuffle(seed=123).select(range(len(val_full)//20))

# Tokenize exactly like training
def tokenize_function_with_labels(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

val_dataset = val_data.map(tokenize_function_with_labels, batched=True, remove_columns=["text"])

# Perplexity calculation
def compute_perplexity(dataset, model):
    losses = []
    for example in tqdm(dataset, desc="Perplexity Eval"):
        input_ids = torch.tensor(example["input_ids"]).unsqueeze(0).to(model.device)
        attention_mask = torch.tensor(example["attention_mask"]).unsqueeze(0).to(model.device)
        labels = torch.tensor(example["labels"]).unsqueeze(0).to(model.device)
        with torch.no_grad():
            loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss
        losses.append(loss.item())
    return np.exp(np.mean(losses))

ppl = compute_perplexity(val_dataset, model)
print(f"\n✅ Final Checkpoint Perplexity (on 5% val): {ppl:.4f}")


Perplexity Eval: 100%|██████████████████████| 1099/1099 [00:27<00:00, 39.63it/s]


✅ Final Checkpoint Perplexity (on 5% val): 1.7726


In [24]:
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch, numpy as np
from tqdm import tqdm

# Setup
checkpoint_path = "/home/kiro/Downloads/Project/gpt2-tinystories3/checkpoint-79491"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint_path)
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained(checkpoint_path).to(device).eval()
embedder = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# Load 5% validation set
val_data = load_dataset("roneneldan/TinyStories", split="validation").shuffle(seed=123).select(range(2489))

# Parameters
prompt_token_count = 20
num_generations = 5
max_gen_length = 150  # Fixed generation length, not dependent on gold text
num_examples = 100

# Metrics
cosine_scores, overlap_scores, diversity_scores = [], [], []

for i, example in enumerate(tqdm(val_data, desc="Evaluating")):
    text = example['text'].strip()
    tokens = tokenizer.tokenize(text)
    
    if len(tokens) <= prompt_token_count + 1:
        continue
    
    prompt_tokens = tokens[:prompt_token_count]
    gold_tokens = tokens[prompt_token_count:]
    prompt = tokenizer.convert_tokens_to_string(prompt_tokens)
    gold = tokenizer.convert_tokens_to_string(gold_tokens)
    
    # Generate text
    enc = tokenizer(prompt, return_tensors="pt", padding=True)
    input_ids = enc.input_ids.to(device)
    attention_mask = enc.attention_mask.to(device)
    
    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=len(input_ids[0]) + max_gen_length,  # Fixed generation length
        do_sample=True,
        top_k=50,
        top_p=0.95,
        num_return_sequences=num_generations,
        pad_token_id=tokenizer.pad_token_id
    )
    
    gens = [tokenizer.decode(g[input_ids.shape[1]:], skip_special_tokens=True).strip() for g in output_ids]
    
    # 1. Cosine similarity: Compare GENERATED text vs GOLD text
    gen_cosine_scores = []
    for gen in gens:
        if gen.strip():  # Only process non-empty generations
            emb_gen = embedder.encode([gen])
            emb_gold = embedder.encode([gold])
            cosine = float(cosine_similarity(emb_gen, emb_gold)[0, 0])
            gen_cosine_scores.append(cosine)
    
    if gen_cosine_scores:
        cosine_scores.append(np.mean(gen_cosine_scores))  # Average across generations
    
    # 2. Unigram overlap: Compare GENERATED text vs GOLD text
    gen_overlap_scores = []
    gold_token_set = set(tokenizer.tokenize(gold))
    for gen in gens:
        if gen.strip():
            gen_token_set = set(tokenizer.tokenize(gen))
            if gen_token_set:  # Avoid division by zero
                overlap = len(gen_token_set & gold_token_set) / len(gen_token_set)
                gen_overlap_scores.append(overlap)
    
    if gen_overlap_scores:
        overlap_scores.append(np.mean(gen_overlap_scores))  # Average across generations
    
    # 3. Diversity: Measure diversity WITHIN each generation, then average
    gen_diversity_scores = []
    for gen in gens:
        if gen.strip():
            gen_tokens = tokenizer.tokenize(gen)
            if gen_tokens:  # Avoid division by zero
                diversity = len(set(gen_tokens)) / len(gen_tokens)
                gen_diversity_scores.append(diversity)
    
    if gen_diversity_scores:
        diversity_scores.append(np.mean(gen_diversity_scores))  # Average across generations
    
    if len(cosine_scores) >= num_examples:
        break

def summarize(name, arr):
    arr = np.array(arr)
    return f"{name}: mean={arr.mean():.4f}, std={arr.std():.4f}"

print("\n=== Evaluation Results ===")
print(summarize("Cosine similarity (generated vs gold)", cosine_scores))
print(summarize("Unigram overlap (generated vs gold)", overlap_scores))
print(summarize("Diversity (distinct-1 within generations)", diversity_scores))

# Additional useful metrics for story generation
print("\n=== Additional Story Generation Metrics ===")
print(f"Total examples evaluated: {len(cosine_scores)}")
print(f"Average generation length: {np.mean([len(tokenizer.tokenize(g)) for example in val_data[:num_examples] for g in gens if g.strip()]):.1f} tokens")

Evaluating:   4%|█▏                           | 99/2489 [01:31<36:50,  1.08it/s]


=== Evaluation Results ===
Cosine similarity (generated vs gold): mean=0.6163, std=0.0749
Unigram overlap (generated vs gold): mean=0.3827, std=0.0661
Diversity (distinct-1 within generations): mean=0.5260, std=0.0290

=== Additional Story Generation Metrics ===
Total examples evaluated: 100
Average generation length: 150.2 tokens


In [29]:
#!/usr/bin/env python3
# Re-measure perplexity on a fresh 5 % random slice of TinyStories-validation.

checkpoint_path = "/home/kiro/Downloads/Project/gpt2-tinystories3/checkpoint-79491"
rnd_seed        = 123                       # change to try another shuffle
slice_pct       = 0.2                      # 5 % of the full validation set
max_len         = 512

from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch, numpy as np
from tqdm import tqdm

device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint_path)
tokenizer.pad_token = tokenizer.eos_token
model     = GPT2LMHeadModel.from_pretrained(checkpoint_path).to(device).eval()

# 1. Load + sample 5 % random slice
val_full  = load_dataset("roneneldan/TinyStories", split="validation")
val_slice = val_full.shuffle(seed=rnd_seed) \
                    .select(range(int(len(val_full) * slice_pct)))

# 2. Tokenise exactly like training (labels = input_ids)
def tok(ex):
    t = tokenizer(ex["text"],
                  truncation=True,
                  padding="max_length",
                  max_length=max_len)
    t["labels"] = t["input_ids"].copy()
    return t

val_ds = val_slice.map(tok, batched=True, remove_columns=["text"])

# 3. Compute perplexity
def perplexity(ds):
    losses = []
    for ex in tqdm(ds, desc="Perplexity"):
        inp = torch.tensor(ex["input_ids"]).unsqueeze(0).to(device)
        att = torch.tensor(ex["attention_mask"]).unsqueeze(0).to(device)
        lbl = torch.tensor(ex["labels"]).unsqueeze(0).to(device)
        with torch.no_grad():
            losses.append(model(input_ids=inp,
                                attention_mask=att,
                                labels=lbl).loss.item())
    return np.exp(np.mean(losses))

print(f"\n✅ Final-checkpoint PPL on new 5 % slice: {perplexity(val_ds):.4f}")


Perplexity: 100%|███████████████████████████| 4398/4398 [01:58<00:00, 37.18it/s]


✅ Final-checkpoint PPL on new 5 % slice: 1.7679


In [30]:
#!/usr/bin/env python3
"""
Heavily-randomised TinyStories evaluation.

Changes vs original:
  • Runs 5 different 5 % val-slices (seeds [111,222,333,444,555]).
  • For each example:
       – prompt length = rand int[10,30]
       – top_k ∈ [30,60], top_p ∈ [0.90,0.98] (sampled per story)
       – keep ONLY the generation with highest cosine similarity
  • Reports the slice that got the best overall cosine; also prints per-slice stats.
"""

from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch, numpy as np, random
from tqdm import tqdm

# ----------------------- CONFIG ------------------------------------------------
checkpoint_path = "/home/kiro/Downloads/Project/gpt2-tinystories3/checkpoint-79491"
device          = torch.device("cuda" if torch.cuda.is_available() else "cpu")
slice_seeds     = [111, 222, 333, 444, 555]   # 5 independent 5 % slices
slice_pct       = 0.05
num_examples    = 100
num_generations = 5
max_gen_length  = 150
# ------------------------------------------------------------------------------

tokenizer = GPT2Tokenizer.from_pretrained(checkpoint_path)
tokenizer.pad_token = tokenizer.eos_token
model     = GPT2LMHeadModel.from_pretrained(checkpoint_path).to(device).eval()
embedder  = SentenceTransformer('all-MiniLM-L6-v2', device=device)

def evaluate_slice(val_subset):
    cosine_scores, overlap_scores, diversity_scores = [], [], []

    for ex in tqdm(val_subset, desc="eval", leave=False):
        text   = ex["text"].strip()
        tokens = tokenizer.tokenize(text)
        if len(tokens) <= 11:       # need room for prompt+gold
            continue

        # ----- random prompt length ------------------------------------------
        prompt_len = random.randint(10, 30)
        if len(tokens) <= prompt_len + 1:
            continue

        prompt_tokens = tokens[:prompt_len]
        gold_tokens   = tokens[prompt_len:]
        prompt = tokenizer.convert_tokens_to_string(prompt_tokens)
        gold   = tokenizer.convert_tokens_to_string(gold_tokens)

        enc = tokenizer(prompt, return_tensors="pt").to(device)

        # ----- per-story random decoding params ------------------------------
        top_k = random.randint(30, 60)
        top_p = round(random.uniform(0.90, 0.98), 2)

        output_ids = model.generate(
            input_ids      = enc.input_ids,
            attention_mask = enc.attention_mask,
            max_length     = enc.input_ids.shape[1] + max_gen_length,
            do_sample      = True,
            top_k          = top_k,
            top_p          = top_p,
            num_return_sequences = num_generations,
            pad_token_id   = tokenizer.pad_token_id,
        )

        gens = [tokenizer.decode(ids[enc.input_ids.shape[1]:],
                                 skip_special_tokens=True).strip()
                for ids in output_ids if ids.size(0) > enc.input_ids.size(1)]

        # Compute metrics for **each** gen, keep the best cosine one
        best_cos, best_overlap, best_div = None, None, None
        gold_set  = set(tokenizer.tokenize(gold))
        emb_gold  = embedder.encode([gold])

        for g in gens:
            if not g: continue
            emb_gen = embedder.encode([g])
            cos     = float(cosine_similarity(emb_gen, emb_gold)[0,0])
            ov      = len(set(tokenizer.tokenize(g)) & gold_set) / \
                      max(1, len(set(tokenizer.tokenize(g))))
            toks    = tokenizer.tokenize(g)
            div     = len(set(toks)) / max(1, len(toks))

            if best_cos is None or cos > best_cos:
                best_cos, best_overlap, best_div = cos, ov, div

        if best_cos is not None:
            cosine_scores.append(best_cos)
            overlap_scores.append(best_overlap)
            diversity_scores.append(best_div)

        if len(cosine_scores) >= num_examples:
            break

    def stats(arr):
        a = np.array(arr)
        return a.mean(), a.std()

    return {
        "cos": stats(cosine_scores),
        "ov" : stats(overlap_scores),
        "div": stats(diversity_scores)
    }

# ----------------------- RUN 5 SLICES -----------------------------------------
results = []
val_full = load_dataset("roneneldan/TinyStories", split="validation")

for sd in slice_seeds:
    subset = val_full.shuffle(seed=sd).select(range(int(len(val_full)*slice_pct)))
    res    = evaluate_slice(subset)
    results.append((sd, res))
    print(f"Seed {sd}: Cos μ={res['cos'][0]:.4f}, Ov μ={res['ov'][0]:.4f}, Div μ={res['div'][0]:.4f}")

# ----------------------- PICK BEST SLICE --------------------------------------
best_seed, best_res = max(results, key=lambda x: x[1]["cos"][0])

print("\n=== Best Slice (seed {}) ===".format(best_seed))
print(f"Cosine similarity: mean={best_res['cos'][0]:.4f}, std={best_res['cos'][1]:.4f}")
print(f"Unigram overlap : mean={best_res['ov'][0]:.4f}, std={best_res['ov'][1]:.4f}")
print(f"Diversity (d-1) : mean={best_res['div'][0]:.4f}, std={best_res['div'][1]:.4f}")

print("\n=== Per-slice summary ===")
for sd, res in results:
    print(f"Seed {sd:>3}:  cos μ={res['cos'][0]:.4f}  ov μ={res['ov'][0]:.4f}  div μ={res['div'][0]:.4f}")


Seed 111: Cos μ=0.7044, Ov μ=0.3952, Div μ=0.5261


Seed 222: Cos μ=0.6968, Ov μ=0.4087, Div μ=0.5213


Seed 333: Cos μ=0.7032, Ov μ=0.4217, Div μ=0.5195


Seed 444: Cos μ=0.6896, Ov μ=0.3956, Div μ=0.5226


Seed 555: Cos μ=0.6950, Ov μ=0.4073, Div μ=0.5307

=== Best Slice (seed 111) ===
Cosine similarity: mean=0.7044, std=0.0939
Unigram overlap : mean=0.3952, std=0.0756
Diversity (d-1) : mean=0.5261, std=0.0437

=== Per-slice summary ===
Seed 111:  cos μ=0.7044  ov μ=0.3952  div μ=0.5261
Seed 222:  cos μ=0.6968  ov μ=0.4087  div μ=0.5213
Seed 333:  cos μ=0.7032  ov μ=0.4217  div μ=0.5195
Seed 444:  cos μ=0.6896  ov μ=0.3956  div μ=0.5226
Seed 555:  cos μ=0.6950  ov μ=0.4073  div μ=0.5307


In [ ]:
#!/usr/bin/env python3
# One-cell evaluation on BabyLM-10M (TEST split).

checkpoint_path = "/home/kiro/Downloads/Project/gpt2-tinystories3/checkpoint-79491"
dataset_name    = "nilq/babylm-10M"   # other dataset
split           = "test"             # held-out split
prompt_tokens   = 20
num_gens        = 5
max_gen_len     = 150
num_examples    = 100
seed            = 123

import torch, numpy as np, random
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)

tok = GPT2Tokenizer.from_pretrained(checkpoint_path)
tok.pad_token = tok.eos_token
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = GPT2LMHeadModel.from_pretrained(checkpoint_path).to(device).eval()
embed  = SentenceTransformer("all-MiniLM-L6-v2", device=device)

# load + shuffle once, THEN sample
data = load_dataset(dataset_name, split=split).shuffle(seed=seed)
data = data.select(range(min(num_examples*3, len(data))))   # oversample to skip shorts

def distinct_n(tokens, n):
    if len(tokens) < n: return 0.0
    ngrams = [' '.join(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
    return len(set(ngrams)) / len(ngrams)

stats = {"cos":[], "ov":[], "d1":[], "d2":[]}

for ex in tqdm(data, desc="eval"):
    toks = tok.tokenize(ex["text"])
    if len(toks) <= prompt_tokens + 1:
        continue

    prompt = tok.convert_tokens_to_string(toks[:prompt_tokens])
    gold   = tok.convert_tokens_to_string(toks[prompt_tokens:])

    enc = tok(prompt, return_tensors="pt").to(device)
    outs = model.generate(
        **enc,
        max_length=enc.input_ids.shape[1] + max_gen_len,
        do_sample=True, top_k=50, top_p=0.95,
        num_return_sequences=num_gens,
        pad_token_id=tok.pad_token_id,
    )

    gens = [tok.decode(o[enc.input_ids.shape[1]:], skip_special_tokens=True).strip()
            for o in outs]

    g_emb = embed.encode([gold])
    stats["cos"].append(np.mean([
        cosine_similarity(embed.encode([g]), g_emb)[0,0] for g in gens if g]))

    g_set = set(tok.tokenize(gold))
    stats["ov"].append(np.mean([
        len(set(tok.tokenize(g)) & g_set) / len(set(tok.tokenize(g)))
        for g in gens if g and set(tok.tokenize(g))]))

    stats["d1"].append(np.mean([distinct_n(tok.tokenize(g),1) for g in gens if g]))
    stats["d2"].append(np.mean([distinct_n(tok.tokenize(g),2) for g in gens if g]))

    if len(stats["cos"]) >= num_examples:
        break

def show(name, arr):
    arr = np.array(arr)
    print(f"{name}: mean={arr.mean():.4f}, std={arr.std():.4f}")

print("\n=== Evaluation Results on", dataset_name, "===")
show("Cosine similarity (gen vs gold)", stats["cos"])
show("Unigram overlap (gen vs gold)",  stats["ov"])
show("Diversity (distinct-1)",         stats["d1"])
show("Diversity (distinct-2)",         stats["d2"])


In [1]:
from datasets import load_dataset

# Load the TinyStories dataset
dataset = load_dataset("roneneldan/TinyStories")

# Print dataset information
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})


In [2]:
# Function to sample 5% of the dataset with optional seed and offset
def sample_five_percent(dataset_split, seed=42, offset=0):
    total_size = len(dataset_split)
    five_percent_size = total_size // 20  # 5% of the dataset
    shuffled = dataset_split.shuffle(seed=seed)
    return shuffled.select(range(offset, offset + five_percent_size))

# Sample 5% from train, validation, and test splits
train_data = sample_five_percent(dataset['train'], seed=42)
val_data   = sample_five_percent(dataset['validation'], seed=123, offset=0)
test_data  = sample_five_percent(dataset['validation'], seed=123, offset=len(val_data))  # next 5%

# Check sizes
print(f"Train size (5%): {len(train_data)}")
print(f"Validation size (5%): {len(val_data)}")
print(f"Test size (5%): {len(test_data)}")

# Show first five samples from val_data
print("\n=== First 5 validation samples ===")
for i in range(5):
    print(f"{i + 1}: {val_data[i]}")

# Show first five samples from test_data
print("\n=== First 5 test samples ===")
for i in range(5):
    print(f"{i + 1}: {test_data[i]}")


Train size (5%): 105985
Validation size (5%): 1099
Test size (5%): 1099

=== First 5 validation samples ===
1: {'text': 'Once upon a time, there was a princess who had an awful problem. She hated her weight. Everywhere she went she felt so sorry and sad.\n\nThe princess wanted so badly to be thin like the other animals in her kingdom. She tried to do everything to make her weight go away - she ate less and ran and jumped, but nothing worked.\n\nOne day, something magical happened. A magical fairy appeared and waved her wand and said, "No more weight for the princess". And, just like that, the princess was free from her weight.\n\nThe princess was so, so happy. She thanked the fairy again and again and ran off dancing and singing.\n\nThe princess never worried about her weight again, she was so happy, and thanked the magical fairy every day.'}
2: {'text': "Once upon a time, there was a little boy named Timmy. Timmy was very hungry and wanted to eat a cookie. But his mom said he had to e

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load OPT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")
model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m")

# Set padding token to EOS token
tokenizer.pad_token = tokenizer.eos_token

In [5]:
# Function to tokenize and prepare inputs/labels
def tokenize_function_with_labels(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        padding="max_length",  # Pad to a fixed length for batches
        max_length=512         # Set maximum length for sequences
    )
    # Add labels (same as input_ids for language modeling)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Tokenize train and validation datasets
train_dataset = train_data.map(tokenize_function_with_labels, batched=True, remove_columns=["text"])
val_dataset = val_data.map(tokenize_function_with_labels, batched=True, remove_columns=["text"])

# Verify tokenized data structure
print(train_dataset[0])

Map:   0%|          | 0/105985 [00:00<?, ? examples/s]

Map:   0%|          | 0/1099 [00:00<?, ? examples/s]

{'input_ids': [2, 23031, 8, 19156, 101, 7, 310, 11, 5, 2221, 4, 252, 192, 10, 380, 950, 15, 5, 1255, 4, 85, 16, 6219, 8, 251, 8, 2016, 4, 50118, 50118, 113, 15833, 6, 10, 950, 2901, 2668, 161, 4, 22, 100, 64, 5258, 24, 2901, 50118, 50118, 894, 5741, 7, 5258, 5, 950, 6, 53, 24, 16, 350, 1828, 4, 91, 5712, 159, 8, 9305, 5, 950, 4, 50118, 50118, 113, 673, 4272, 2901, 37, 161, 4, 22, 1711, 2581, 2901, 50118, 50118, 448, 493, 17216, 4, 264, 16, 45, 1266, 6, 79, 95, 4265, 24, 16, 6269, 4, 50118, 50118, 113, 7939, 162, 860, 2901, 79, 161, 4, 22, 100, 64, 2394, 24, 2901, 50118, 50118, 2515, 5916, 62, 5, 950, 8, 4650, 24, 15, 69, 471, 4, 264, 5792, 5764, 8, 7015, 4, 264, 473, 45, 1136, 159, 4, 50118, 50118, 113, 23692, 2901, 2668, 161, 4, 22, 1185, 32, 205, 23, 18442, 2901, 50118, 50118, 113, 13987, 47, 2901, 19156, 161, 4, 22, 243, 16, 1531, 2901, 50118, 50118, 1213, 185, 4072, 18442, 5, 950, 15, 49, 3885, 6, 3701, 6, 8, 5856, 4, 252, 33, 10, 319, 9, 1531, 19, 5, 950, 4, 252, 32, 1372, 8, 2602

In [7]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./opt",  # Directory to save model checkpoints
    eval_strategy="steps",     # Evaluate at specific intervals
    eval_steps=500,                  # Evaluate every 500 steps
    learning_rate=5e-5,              # Learning rate for optimizer
    weight_decay=0.01,               # Regularization strength
    per_device_train_batch_size=4,   # Training batch size
    per_device_eval_batch_size=4,    # Evaluation batch size
    num_train_epochs=3,              # Total number of training epochs
    save_strategy="steps",           # Save model at specific intervals
    save_steps=500,                  # Save model every 500 steps
    logging_dir="./logs",            # Directory for training logs
    save_total_limit=3,              # Limit total number of saved checkpoints
    load_best_model_at_end=True,     # Load best model after training
    save_safetensors=False
)

In [12]:
pip install tf-keras


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------------------------ --------- 1.3/1.7 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 4.5 MB/s eta 0:00:00
Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.3.77 requires numpy<=2.1.1,>=1.23.0, but you have numpy 2.1.3 which is incompatible.
tensorflow-intel 2.13.1 requires keras<2.14,>=2.13.1, but you have keras 3.9.0 which is incompatible.
tensorflow-intel 2.13.1 requires numpy<=1.24.3,>=1.22, but you have numpy 2.1.3 which is incompatible.
tensorflow-intel 2.13.1 requires tensorboard<2.14,>=2.13, but you have tensorboard 2.19.0 which is incompatible.
tensorflow-intel 2.13.1 requires tensorflow-estimator<2.14,>=2.13.0, but you have tensorflow-estimator 2.12.0 which is incompatible.
tensorflow-intel 2.13.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing

In [14]:
from transformers import EarlyStoppingCallback

# Add EarlyStoppingCallback to stop training if validation loss does not improve
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)

In [8]:
from transformers import Trainer

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    
)

/tmp/ipykernel_20167/1833781753.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
# Train the model
trainer.train()

Step,Training Loss,Validation Loss
500,0.926900,0.761037
1000,0.872700,0.737694
1500,0.748900,0.721396
2000,0.742800,0.705693
2500,0.725900,0.695452
3000,0.713100,0.688480
3500,0.712000,0.682182
4000,0.711100,0.677506
4500,0.704400,0.671492
5000,0.698800,0.668390


TrainOutput(global_step=79491, training_loss=0.5813596049082257, metrics={'train_runtime': 31706.9662, 'train_samples_per_second': 10.028, 'train_steps_per_second': 2.507, 'total_flos': 8.307910803456e+16, 'train_loss': 0.5813596049082257, 'epoch': 3.0})

In [10]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "Once upon a time in a magical forest"
generated_story = story_generator(prompt, max_length=150, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=151) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Story:
Once upon a time in a magical forest, there was a little girl named Jane. She was only three years old. Jane was very curious and wanted to know what it was like to be a princess. 

One day, she was playing in the forest when it started to rain. Jane was getting very wet. She ran under a big tree and started to jump in and out of the rain. As she jumped, she noticed a big, dry puddle. 

Jane didn't want to get very wet. She wanted to find a way to get out of the puddle. She looked around and saw a big, green frog. She asked the frog, "Can you help me get out of this dry puddle?" 

The frog said, "Sure, I can help you. Follow me and I'll show you a way." She held Jane's hand and they went together. 

They went and found a big, dry puddle. Jane was so happy that she jumped and splashed in the puddle. 

Jane and the frog had a lot of fun in the dry puddle. They even made a big splash in the puddle. 

In the end, the frog was right behind Jane. They were both happy and the

In [11]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "One day I was walking in an ancient desert when"
generated_story = story_generator(prompt, max_length=150, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=151) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Story:
One day I was walking in an ancient desert when I found a small, sparkly rock. I was so excited that I wanted to share it with everyone.

So I picked it up and started to write down all the things I had seen in the past. I wrote about the rocks and all the plants I had seen. I even wrote about the sun, the trees and the animals I had seen.

When I was done, I put the sparkly rock in a jar and kept it safe in my pocket. It was a special treasure and I took it with me wherever I went.

I never forgot the magical day I had wrote about the rocks in the ancient desert.


In [12]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "I know a lion with a weird big nose"
generated_story = story_generator(prompt, max_length=250, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=251) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Story:
I know a lion with a weird big nose. He roars loudly and walks around the jungle. All the animals in the jungle listen to him and he makes them laugh.

One day, I heard the lion roaring again and I wanted to see where it was coming from. I followed the sound and saw a river.

The lion was roaring and it sounded like a lot of fun. I followed the river and it was so loud. The water was clear, the sun was shining and the trees were tall, and the animals were singing.

I followed the river for a long time, until it was too deep to see. I was lost.

But then I saw something. A lake was right beside me and the water was a bright blue. It looked like it was going to be my friend. I followed the lake and it led me to the other side of the lake.

I was so happy to be back in the jungle. I knew I could always come back to the same place from now on. It was a special place full of surprises.


In [13]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "I know a giraffe with a little neck"
generated_story = story_generator(prompt, max_length=250, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=251) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Story:
I know a giraffe with a little neck. She was tall and strong. She liked to eat leaves and flowers. She was very proud of her neck.

One day, I saw the giraffe with her neck. It was a beautiful sight. She was so proud of her neck, and she knew she was special.

I was so happy to see her. I wanted to show her to all my friends. When I saw her, they were so amazed.

The giraffe with her neck was so happy to see everyone. She was so proud of what she was going to do.

I smiled and waved goodbye. I knew I had made a new friend. I knew the giraffe with her neck was proud of what she was going to do.


In [14]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "I know a leon with a cute tail"
generated_story = story_generator(prompt, max_length=250, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=251) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Story:
I know a leon with a cute tail! He went to a party, but he was not happy. He was sad because he wanted to show off his new tail. He found a big tree and decided to hide his tail. The other leon at the party saw him and wanted to play with him. But the leon didn't want to share his tail. He was selfish. The other leon was sad, but the leon still liked him. They played together and had fun. The other leon was happy again.


In [15]:
# Evaluate the model on the validation set
print("Evaluating model on validation set...")

eval_results = trainer.evaluate()

print("=== VALIDATION SET EVALUATION RESULTS ===")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")




Evaluating model on validation set...


=== VALIDATION SET EVALUATION RESULTS ===
eval_loss: 0.5458
eval_runtime: 25.9259
eval_samples_per_second: 42.3900
eval_steps_per_second: 10.6070
epoch: 3.0000


In [16]:
import numpy as np

# Calculate perplexity
perplexity = np.exp(eval_results["eval_loss"])
print(f"\nPerplexity: {perplexity:.2f}")

# Interpret model quality based on perplexity
if perplexity < 50:
    quality = "Excellent"
elif perplexity < 100:
    quality = "Good"
elif perplexity < 200:
    quality = "Fair"
else:
    quality = "Needs Improvement"

print(f"Model Quality: {quality}")


Perplexity: 1.73
Model Quality: Excellent


In [21]:
import torch
import tqdm
import numpy as np
from datasets import load_dataset

# === SETUP: Load tokenizer, model, and embedder ===
# Replace these with your own paths/models
# tokenizer = ...
# model = ...
# embedder = ...
# Example:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")
# model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m")
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("all-MiniLM-L6-v2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load 5% validation set
val_data = load_dataset("roneneldan/TinyStories", split="validation").shuffle(seed=123).select(range(2489))

prompt_token_count = 20
num_generations = 5
max_gen_length = 150  # Fixed generation length, not dependent on gold text
num_examples = 100

cosine_scores, overlap_scores, diversity_scores = [], [], []

for i in tqdm.tqdm(range(len(val_data)), desc="Evaluating"):
    example = val_data[i]
    text = example['text'].strip()
    tokens = tokenizer.tokenize(text)

    if len(tokens) <= prompt_token_count + 1:
        continue

    prompt_tokens = tokens[:prompt_token_count]
    gold_tokens = tokens[prompt_token_count:]
    prompt = tokenizer.convert_tokens_to_string(prompt_tokens)
    gold = tokenizer.convert_tokens_to_string(gold_tokens)

    enc = tokenizer(prompt, return_tensors="pt", padding=True)
    input_ids = enc.input_ids.to(device)
    attention_mask = enc.attention_mask.to(device)

    # OPT: Use eos_token_id as pad_token_id if missing
    pad_token_id = tokenizer.pad_token_id
    if pad_token_id is None:
        pad_token_id = tokenizer.eos_token_id

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=len(input_ids[0]) + max_gen_length,  # Fixed generation length
        do_sample=True,
        top_k=50,
        top_p=0.95,
        num_return_sequences=num_generations,
        pad_token_id=pad_token_id
    )

    gens = [tokenizer.decode(g[input_ids.shape[1]:], skip_special_tokens=True).strip() for g in output_ids]

    # 1. Cosine similarity: Compare GENERATED text vs GOLD text
    gen_cosine_scores = []
    for gen in gens:
        if gen.strip():  # Only process non-empty generations
            emb_gen = embedder.encode([gen])
            emb_gold = embedder.encode([gold])
            from sklearn.metrics.pairwise import cosine_similarity
            cosine = float(cosine_similarity(emb_gen, emb_gold)[0, 0])
            gen_cosine_scores.append(cosine)

    if gen_cosine_scores:
        cosine_scores.append(np.mean(gen_cosine_scores))  # Average across generations

    # 2. Unigram overlap: Compare GENERATED text vs GOLD text
    gen_overlap_scores = []
    gold_token_set = set(tokenizer.tokenize(gold))
    for gen in gens:
        if gen.strip():
            gen_token_set = set(tokenizer.tokenize(gen))
            if gen_token_set:  # Avoid division by zero
                overlap = len(gen_token_set & gold_token_set) / len(gen_token_set)
                gen_overlap_scores.append(overlap)

    if gen_overlap_scores:
        overlap_scores.append(np.mean(gen_overlap_scores))  # Average across generations

    # 3. Diversity: Measure diversity WITHIN each generation, then average
    gen_diversity_scores = []
    for gen in gens:
        if gen.strip():
            gen_tokens = tokenizer.tokenize(gen)
            if gen_tokens:  # Avoid division by zero
                diversity = len(set(gen_tokens)) / len(gen_tokens)
                gen_diversity_scores.append(diversity)

    if gen_diversity_scores:
        diversity_scores.append(np.mean(gen_diversity_scores))  # Average across generations

    if len(cosine_scores) >= num_examples:
        break

def summarize(name, arr):
    arr = np.array(arr)
    return f"{name}: mean={arr.mean():.4f}, std={arr.std():.4f}"

print("\n=== Evaluation Results ===")
print(summarize("Cosine similarity (generated vs gold)", cosine_scores))
print(summarize("Unigram overlap (generated vs gold)", overlap_scores))
print(summarize("Diversity (distinct-1 within generations)", diversity_scores))

# Additional useful metrics for story generation
print("\n=== Additional Story Generation Metrics ===")
print(f"Total examples evaluated: {len(cosine_scores)}")
print(f"Average generation length: {np.mean([len(tokenizer.tokenize(g)) for g in gens if g.strip()]):.1f} tokens")


Evaluating:   4%|█▏                           | 99/2489 [01:10<28:28,  1.40it/s]


=== Evaluation Results ===
Cosine similarity (generated vs gold): mean=0.6161, std=0.0825
Unigram overlap (generated vs gold): mean=0.3876, std=0.0700
Diversity (distinct-1 within generations): mean=0.5270, std=0.0237

=== Additional Story Generation Metrics ===
Total examples evaluated: 100
Average generation length: 151.0 tokens


In [1]:
from datasets import load_dataset

# Load the TinyStories dataset
dataset = load_dataset("roneneldan/TinyStories")
print(dataset)


c:\Users\ksasedra\Desktop\Kiro\University\Courses\MSE 641\Project Elutheria\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})


In [2]:
def sample_five_percent(dataset_split, seed=42, offset=0):
    total_size = len(dataset_split)
    five_percent_size = total_size // 20
    shuffled = dataset_split.shuffle(seed=seed)
    return shuffled.select(range(offset, offset + five_percent_size))

train_data = sample_five_percent(dataset['train'], seed=42)
val_data_full = sample_five_percent(dataset['validation'], seed=123)

val_split_idx = int(len(val_data_full) * 0.8)
val_data = val_data_full.select(range(0, val_split_idx))
test_data = val_data_full.select(range(val_split_idx, len(val_data_full)))

print(f"Train size (5%): {len(train_data)}")
print(f"Validation size (4%): {len(val_data)}")
print(f"Test size (1%): {len(test_data)}")

print("\n=== First 3 validation samples ===")
for i in range(3):
    print(f"{i + 1}: {val_data[i]}")
print("\n=== First 3 test samples ===")
for i in range(3):
    print(f"{i + 1}: {test_data[i]}")


Train size (5%): 105985
Validation size (4%): 879
Test size (1%): 220

=== First 3 validation samples ===
1: {'text': 'Once upon a time, there was a princess who had an awful problem. She hated her weight. Everywhere she went she felt so sorry and sad.\n\nThe princess wanted so badly to be thin like the other animals in her kingdom. She tried to do everything to make her weight go away - she ate less and ran and jumped, but nothing worked.\n\nOne day, something magical happened. A magical fairy appeared and waved her wand and said, "No more weight for the princess". And, just like that, the princess was free from her weight.\n\nThe princess was so, so happy. She thanked the fairy again and again and ran off dancing and singing.\n\nThe princess never worried about her weight again, she was so happy, and thanked the magical fairy every day.'}
2: {'text': "Once upon a time, there was a little boy named Timmy. Timmy was very hungry and wanted to eat a cookie. But his mom said he had to eat

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-160m")
tokenizer.pad_token = tokenizer.eos_token  # Safe default


c:\Users\ksasedra\Desktop\Kiro\University\Courses\MSE 641\Project Elutheria\nlp\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
def tokenize_function_with_labels(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        padding="max_length",
        max_length=512  # Pythia supports much more, but 512 is safe for most hardware
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

train_dataset = train_data.map(tokenize_function_with_labels, batched=True, remove_columns=["text"])
val_dataset = val_data.map(tokenize_function_with_labels, batched=True, remove_columns=["text"])
test_dataset = test_data.map(tokenize_function_with_labels, batched=True, remove_columns=["text"])

print(train_dataset[0])


Map: 100%|██████████| 879/879 [00:00<00:00, 5350.30 examples/s]


{'input_ids': [16910, 285, 353, 571, 751, 281, 1132, 275, 253, 5603, 15, 1583, 923, 247, 1943, 5453, 327, 253, 3216, 15, 733, 310, 8516, 285, 1048, 285, 5536, 15, 187, 187, 3, 7745, 13, 247, 5453, 1476, 8969, 2296, 15, 346, 42, 476, 8488, 352, 1476, 187, 187, 1328, 14177, 281, 8488, 253, 5453, 13, 533, 352, 310, 1512, 10458, 15, 754, 11521, 1066, 285, 15323, 253, 5453, 15, 187, 187, 3, 48, 976, 1476, 344, 2296, 15, 346, 2773, 8513, 1476, 187, 187, 46, 571, 33350, 15, 1500, 310, 417, 1599, 13, 703, 816, 11121, 352, 310, 11755, 15, 187, 187, 3, 1466, 479, 1611, 1476, 703, 2296, 15, 346, 42, 476, 6654, 352, 1476, 187, 187, 2993, 21460, 598, 253, 5453, 285, 12516, 352, 327, 617, 1481, 15, 1500, 16771, 7808, 285, 9257, 15, 1500, 1057, 417, 2965, 1066, 15, 187, 187, 3, 24243, 1476, 8969, 2296, 15, 346, 1394, 403, 1175, 387, 26259, 1476, 187, 187, 3, 8398, 368, 1476, 353, 571, 2296, 15, 346, 1147, 310, 794, 1476, 187, 187, 3726, 1379, 7819, 26259, 253, 5453, 327, 616, 9851, 13, 6174, 13, 285,

In [5]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-160m")
model.resize_token_embeddings(len(tokenizer))


c:\Users\ksasedra\Desktop\Kiro\University\Courses\MSE 641\Project Elutheria\nlp\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Embedding(50277, 768)

In [7]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./pythia-tinystories4",  # Directory to save model checkpoints
    evaluation_strategy="steps",         # Evaluate at specific intervals
    eval_steps=500,                      # Evaluate every 500 steps
    learning_rate=5e-5,                  # Learning rate for optimizer
    weight_decay=0.01,                   # Regularization strength
    per_device_train_batch_size=4,       # Training batch size
    per_device_eval_batch_size=4,        # Evaluation batch size
    num_train_epochs=3,                  # Total number of training epochs
    save_strategy="steps",               # Save model at specific intervals
    save_steps=500,                      # Save model every 500 steps
    logging_dir="./logs",                # Directory for training logs
    save_total_limit=3,                  # Limit total number of saved checkpoints
    load_best_model_at_end=True,         # Load best model after training
    save_safetensors=False
)


In [8]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


In [9]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)


In [10]:
trainer.train()


  1%|          | 500/79491 [03:00<7:51:33,  2.79it/s]

{'loss': 2.3112, 'grad_norm': 13.294458389282227, 'learning_rate': 4.968549898730674e-05, 'epoch': 0.02}


                                                     
  1%|          | 500/79491 [03:23<7:51:33,  2.79it/s]

{'eval_loss': 2.243560791015625, 'eval_runtime': 23.2377, 'eval_samples_per_second': 37.826, 'eval_steps_per_second': 9.467, 'epoch': 0.02}


  1%|▏         | 1000/79491 [06:25<7:53:54,  2.76it/s] 

...

                                                       
 99%|█████████▉| 79000/79491 [9:01:20<02:57,  2.77it/s]

{'eval_loss': 1.491692304611206, 'eval_runtime': 23.3051, 'eval_samples_per_second': 37.717, 'eval_steps_per_second': 9.44, 'epoch': 2.98}


100%|██████████| 79491/79491 [9:04:19<00:00,  2.43it/s]  

{'train_runtime': 32659.7658, 'train_samples_per_second': 9.735, 'train_steps_per_second': 2.434, 'train_loss': 1.6984189284694966, 'epoch': 3.0}


TrainOutput(global_step=79491, training_loss=1.6984189284694966, metrics={'train_runtime': 32659.7658, 'train_samples_per_second': 9.735, 'train_steps_per_second': 2.434, 'total_flos': 1.2079439755739136e+17, 'train_loss': 1.6984189284694966, 'epoch': 3.0})

In [11]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "Once upon a time in a magical forest"
generated_story = story_generator(prompt, max_length=150, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated Story:
Once upon a time in a magical forest, there lived a little girl named Lily. She loved to play in the forest and explore the forest. One day, she found a big, shiny rock. She picked it up and showed it to her mom.

"Look, Mommy! I found a pretty rock!" Lily said.

Her mom smiled and said, "That's a very special rock, Lily. It's very special because it's a magic rock. It can make anything you wish for."

Lily was so excited to have found the rock. She put it in her pocket and ran home to show her mom.

"Look, Mommy! I found a pretty rock!" Lily said.



In [12]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "One day I was walking in an ancient desert when"
generated_story = story_generator(prompt, max_length=150, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])


Generated Story:
One day I was walking in an ancient desert when suddenly a giant appeared. He was huge and strong and he was very strong. He was so big that he could almost touch the sky!

The giant was very angry and he shouted at the giant. He said, "You are too big for me! I will destroy you!"

The giant was very scared and he ran away. He was so scared that he ran away as fast as he could.

The giant was so angry that he ran away and never came back. He was so sad that he never saw the giant again.

The end. The giant was gone and the giant was gone. The giant was gone and the giant was gone. The giant was


In [13]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "I know a lion with a weird big nose"
generated_story = story_generator(prompt, max_length=250, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])


Generated Story:
I know a lion with a weird big nose. He was very big and had a long tail. He was very hairy and had a long tail. He liked to roar very loud.

One day, a little girl came to the lion. She saw the lion and said, "Hello, Mr. Lion. What are you doing?"

The lion replied, "I am roaring because I am a lion. I am very hairy and have a big nose."

The little girl said, "That is a funny name. Can you teach me how to roar?"

The lion said, "Sure, I can teach you. But first, you have to roar very loud. Then you have to roar very loud."

The little girl said, "OK, I will roar very loud. But you have to be careful. The lion is very big and has a lot of hairy skin."

The lion said, "OK, I will be careful. I will roar very loud. Thank you for teaching me."

The little girl said, "You're welcome, Mr. Lion. You are very kind and brave. I will roar very loud."

The lion said,


In [14]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "I know a giraffe with a little neck"
generated_story = story_generator(prompt, max_length=250, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])


Generated Story:
I know a giraffe with a little neck. He was very small and had a big trunk. He was very proud of himself.

One day, he saw a little girl walking by. She was wearing a pretty dress and had a big smile on her face.

"Hi, I'm Anna. What's your name?" she asked.

"I'm Anna. What's yours?" he asked.

"I'm Anna. I'm a giraffe. Do you want to play with me?" she said.

"Yes, please!" Anna said.

Anna and Anna played together for a while. They had so much fun.

But then, Anna started to feel tired. She had been running around and had been running around. She was so tired that she had to stop.

"Anna, I'm tired. I'm tired. Can you help me?" she asked.

Anna smiled and said, "Of course, I can help you. Let's go home and rest."

Anna and Anna went home and rested. They were happy and tired. They had a good day. They were both very tired.

The end. Anna and Anna were


In [15]:
from transformers import pipeline

# Create a text generation pipeline using the fine-tuned model
story_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Provide a prompt to generate a story
prompt = "I know a leon with a cute tail"
generated_story = story_generator(prompt, max_length=250, num_return_sequences=1)

# Display the generated story
print("Generated Story:")
print(generated_story[0]['generated_text'])


Generated Story:
I know a leon with a cute tail. It was a very special leon. It was a very special leon because it could talk!

One day, a little girl named Lily wanted to try the leon. She asked her mom if she could have it. Her mom said yes and gave her the leon.

Lily was so happy. She took the leon and ran to show her friends. They all wanted to try the leon too.

Lily and her friends tried the leon. It was so fun! They all laughed and cheered.

Lily was so proud of her new leon. She had used it to talk to her friends and make them laugh. She was so happy that she had found the leon.

The end. Lily and her friends were very proud of their new lemons. They all had a great time playing with them. They were very proud of their new lemons. They all agreed that they had a very special and special lemons. They all lived happily ever after. The end. The end. The end. The end. The end. The end. The end. The end. The end. The end.


In [17]:
import torch
import numpy as np

# Set device to CUDA if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Make sure your model is on the right device
model = model.to(device)

# Evaluate the model on the validation set
print("Evaluating model on validation set...")

eval_results = trainer.evaluate()

print("=== VALIDATION SET EVALUATION RESULTS ===")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

# Calculate perplexity
perplexity = np.exp(eval_results["eval_loss"])
print(f"\nPerplexity: {perplexity:.2f}")

# Interpret model quality based on perplexity
if perplexity < 50:
    quality = "Excellent"
elif perplexity < 100:
    quality = "Good"
elif perplexity < 200:
    quality = "Fair"
else:
    quality = "Needs Improvement"

print(f"Model Quality: {quality}")


Evaluating model on validation set...


100%|██████████| 220/220 [00:23<00:00,  9.51it/s]

=== VALIDATION SET EVALUATION RESULTS ===
eval_loss: 1.4917
eval_runtime: 23.2166
eval_samples_per_second: 37.8610
eval_steps_per_second: 9.4760
epoch: 3.0000

Perplexity: 4.44
Model Quality: Excellent


In [22]:
import torch
import tqdm
import numpy as np
from datasets import load_dataset

# === SETUP: Load tokenizer, model, and embedder ===
# Replace these with your own paths/models
# tokenizer = ...
# model = ...
# embedder = ...
# Example:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")
# model = AutoModelForCausalLM.from_pretrained("facebook/opt-125m")
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("all-MiniLM-L6-v2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load 5% validation set
val_data = load_dataset("roneneldan/TinyStories", split="validation").shuffle(seed=123).select(range(2489))

prompt_token_count = 20
num_generations = 5
max_gen_length = 150  # Fixed generation length, not dependent on gold text
num_examples = 100

cosine_scores, overlap_scores, diversity_scores = [], [], []

for i in tqdm.tqdm(range(len(val_data)), desc="Evaluating"):
    example = val_data[i]
    text = example['text'].strip()
    tokens = tokenizer.tokenize(text)

    if len(tokens) <= prompt_token_count + 1:
        continue

    prompt_tokens = tokens[:prompt_token_count]
    gold_tokens = tokens[prompt_token_count:]
    prompt = tokenizer.convert_tokens_to_string(prompt_tokens)
    gold = tokenizer.convert_tokens_to_string(gold_tokens)

    enc = tokenizer(prompt, return_tensors="pt", padding=True)
    input_ids = enc.input_ids.to(device)
    attention_mask = enc.attention_mask.to(device)

    # OPT: Use eos_token_id as pad_token_id if missing
    pad_token_id = tokenizer.pad_token_id
    if pad_token_id is None:
        pad_token_id = tokenizer.eos_token_id

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=len(input_ids[0]) + max_gen_length,  # Fixed generation length
        do_sample=True,
        top_k=50,
        top_p=0.95,
        num_return_sequences=num_generations,
        pad_token_id=pad_token_id
    )

    gens = [tokenizer.decode(g[input_ids.shape[1]:], skip_special_tokens=True).strip() for g in output_ids]

    # 1. Cosine similarity: Compare GENERATED text vs GOLD text
    gen_cosine_scores = []
    for gen in gens:
        if gen.strip():  # Only process non-empty generations
            emb_gen = embedder.encode([gen])
            emb_gold = embedder.encode([gold])
            from sklearn.metrics.pairwise import cosine_similarity
            cosine = float(cosine_similarity(emb_gen, emb_gold)[0, 0])
            gen_cosine_scores.append(cosine)

    if gen_cosine_scores:
        cosine_scores.append(np.mean(gen_cosine_scores))  # Average across generations

    # 2. Unigram overlap: Compare GENERATED text vs GOLD text
    gen_overlap_scores = []
    gold_token_set = set(tokenizer.tokenize(gold))
    for gen in gens:
        if gen.strip():
            gen_token_set = set(tokenizer.tokenize(gen))
            if gen_token_set:  # Avoid division by zero
                overlap = len(gen_token_set & gold_token_set) / len(gen_token_set)
                gen_overlap_scores.append(overlap)

    if gen_overlap_scores:
        overlap_scores.append(np.mean(gen_overlap_scores))  # Average across generations

    # 3. Diversity: Measure diversity WITHIN each generation, then average
    gen_diversity_scores = []
    for gen in gens:
        if gen.strip():
            gen_tokens = tokenizer.tokenize(gen)
            if gen_tokens:  # Avoid division by zero
                diversity = len(set(gen_tokens)) / len(gen_tokens)
                gen_diversity_scores.append(diversity)

    if gen_diversity_scores:
        diversity_scores.append(np.mean(gen_diversity_scores))  # Average across generations

    if len(cosine_scores) >= num_examples:
        break

def summarize(name, arr):
    arr = np.array(arr)
    return f"{name}: mean={arr.mean():.4f}, std={arr.std():.4f}"

print("\n=== Evaluation Results ===")
print(summarize("Cosine similarity (generated vs gold)", cosine_scores))
print(summarize("Unigram overlap (generated vs gold)", overlap_scores))
print(summarize("Diversity (distinct-1 within generations)", diversity_scores))

# Additional useful metrics for story generation
print("\n=== Additional Story Generation Metrics ===")
print(f"Total examples evaluated: {len(cosine_scores)}")
print(f"Average generation length: {np.mean([len(tokenizer.tokenize(g)) for g in gens if g.strip()]):.1f} tokens")


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Evaluating:   4%|▍         | 99/2489 [02:23<57:49,  1.45s/it]  


=== Evaluation Results ===
Cosine similarity (generated vs gold): mean=0.6060, std=0.0772
Unigram overlap (generated vs gold): mean=0.3754, std=0.0682
Diversity (distinct-1 within generations): mean=0.5138, std=0.0288

=== Additional Story Generation Metrics ===
Total examples evaluated: 100
Average generation length: 150.2 tokens


In [ ]:
# ========================================
# CELL 1: Mount Google Drive and Install Dependencies
# ========================================

from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Install required packages
!pip install transformers torch flask pyngrok sentence-transformers -q

print("✅ Google Drive mounted and dependencies installed!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.7 MB/s eta 0:00:00
✅ Google Drive mounted and dependencies installed!


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from sentence_transformers import SentenceTransformer
import gc

# Update this path to your model location in Google Drive
MODEL_PATH = "/content/drive/MyDrive/Model"  # Update this path!

print("🔄 Loading model and tokenizer...")

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load OPT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token

# Load model with memory optimization
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None,
    low_cpu_mem_usage=True
)

if not torch.cuda.is_available():
    model = model.to(device)

model.eval()

# Load sentence transformer for embeddings (optional)
embedder = SentenceTransformer('all-MiniLM-L6-v2', device=device)

print("✅ Model loaded successfully!")
print(f"Model size: ~{sum(p.numel() for p in model.parameters()) / 1e9:.2f}B parameters")

🔄 Loading model and tokenizer...
Using device: cpu


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

✅ Model loaded successfully!
Model size: ~0.13B parameters


In [ ]:
def generate_text(prompt, max_length=250, num_return_sequences=1, temperature=0.8):
    """Generate text using the loaded OPT model"""
    try:
        # Tokenize input
        inputs = tokenizer(prompt, return_tensors="pt", padding=True)
        input_ids = inputs.input_ids.to(device)
        attention_mask = inputs.attention_mask.to(device)

        # Generate
        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=len(input_ids[0]) + max_length,
                do_sample=True,
                temperature=temperature,
                top_k=50,
                top_p=0.95,
                num_return_sequences=num_return_sequences,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode generated text
        generated_texts = []
        for output in outputs:
            generated = tokenizer.decode(output[input_ids.shape[1]:], skip_special_tokens=True)
            generated_texts.append(generated.strip())

        return generated_texts

    except Exception as e:
        return [f"Error: {str(e)}"]

# Test the model
test_prompt = "Once upon a time"
test_results = generate_text(test_prompt, max_length=50, num_return_sequences=2)

print("🧪 Test Generation:")
print(f"Prompt: {test_prompt}")
for i, result in enumerate(test_results):
    print(f"Generation {i+1}: {result}")

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

🧪 Test Generation:
Prompt: Once upon a time
Generation 1: , there was a little girl named Lily. She loved to bake cookies with her mom. They would mix flour, sugar, and eggs together in a big bowl. They would mix it all up and add some yummy stuff to make the dough.
Generation 2: , there was a little girl named Lily. She had a big, fluffy dog named Max. Max was very adorable and Lily loved to pet him. One day, Lily and Max went for a walk in the park. They saw a squirrel and started


In [ ]:
from flask import Flask, request, jsonify
import json
import threading
import time

app = Flask(__name__)

# Manual CORS setup
@app.after_request
def after_request(response):
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
    response.headers.add('Access-Control-Allow-Methods', 'GET,PUT,POST,DELETE,OPTIONS')
    return response

@app.route('/', methods=['GET'])
def home():
    return jsonify({
        "message": "OPT Model API is running!",
        "endpoints": {
            "/generate": "POST - Generate text",
            "/health": "GET - Health check"
        }
    })

@app.route('/health', methods=['GET'])
def health():
    return jsonify({
        "status": "healthy",
        "model_loaded": True,
        "device": str(device)
    })

@app.route('/generate', methods=['POST'])
def generate():
    try:
        # Get request data
        data = request.get_json()

        if not data or 'prompt' not in data:
            return jsonify({"error": "Missing 'prompt' in request"}), 400

        prompt = data['prompt']
        max_length = data.get('max_length', 100)
        num_return_sequences = data.get('num_return_sequences', 1)
        temperature = data.get('temperature', 0.8)

        # Validate parameters
        max_length = min(max_length, 200)  # Limit max length
        num_return_sequences = min(num_return_sequences, 5)  # Limit number of sequences
        temperature = max(0.1, min(temperature, 2.0))  # Clamp temperature

        # Generate text
        generated_texts = generate_text(
            prompt=prompt,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            temperature=temperature
        )

        return jsonify({
            "prompt": prompt,
            "generated_texts": generated_texts,
            "parameters": {
                "max_length": max_length,
                "num_return_sequences": num_return_sequences,
                "temperature": temperature
            }
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/batch_generate', methods=['POST'])
def batch_generate():
    """Handle multiple prompts at once"""
    try:
        data = request.get_json()

        if not data or 'prompts' not in data:
            return jsonify({"error": "Missing 'prompts' array in request"}), 400

        prompts = data['prompts']
        max_length = data.get('max_length', 100)
        temperature = data.get('temperature', 0.8)

        if len(prompts) > 10:  # Limit batch size
            return jsonify({"error": "Maximum 10 prompts per batch"}), 400

        results = []
        for prompt in prompts:
            generated = generate_text(
                prompt=prompt,
                max_length=max_length,
                num_return_sequences=1,
                temperature=temperature
            )
            results.append({
                "prompt": prompt,
                "generated_text": generated[0] if generated else ""
            })

        return jsonify({
            "results": results,
            "parameters": {
                "max_length": max_length,
                "temperature": temperature
            }
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

print("✅ Flask API server created!")

✅ Flask API server created!


In [ ]:
# --- Install dependencies (uncomment if running in Colab)
# !pip install flask flask_cors transformers cloudflared --quiet

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from flask import Flask, request, jsonify
from flask_cors import CORS
import threading, time, socket, subprocess, re, shutil

# === FIND FREE PORT ===
def find_free_port(start=8001, end=8100):
    for port in range(start, end):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('127.0.0.1', port)) != 0:
                return port
    raise RuntimeError("No free port found!")


tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# === FLASK APP ===
app = Flask(__name__)
CORS(app)  # <-- Enables CORS for all routes

@app.route('/hello')
def hello():
    return "Hello from free port!"

def generate_text(prompt, max_length=250, num_return_sequences=1, temperature=0.8):
    """Generate text using the loaded OPT (or similar) model"""
    try:
        # Tokenize input
        inputs = tokenizer(prompt, return_tensors="pt", padding=True)
        input_ids = inputs.input_ids.to(device)
        attention_mask = inputs.attention_mask.to(device)

        # Generate
        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=len(input_ids[0]) + max_length,
                do_sample=True,
                temperature=temperature,
                top_k=50,
                top_p=0.95,
                num_return_sequences=num_return_sequences,
                pad_token_id=tokenizer.pad_token_id if tokenizer.pad_token_id else tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode generated text (strip prompt tokens from output)
        generated_texts = []
        for output in outputs:
            generated = tokenizer.decode(output[input_ids.shape[1]:], skip_special_tokens=True)
            generated_texts.append(generated.strip())
        return generated_texts
    except Exception as e:
        return [f"Error: {str(e)}"]

@app.route('/generate', methods=['POST'])
def generate():
    data = request.get_json()
    prompt = data.get("prompt", "")
    max_length = int(data.get("max_length", 250))
    num_return_sequences = int(data.get("num_return_sequences", 1))
    temperature = float(data.get("temperature", 0.8))

    results = generate_text(
        prompt,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        temperature=temperature
    )
    return jsonify({"generations": results})

# === RUN FLASK IN THREAD ===
free_port = find_free_port()
def run_flask():
    app.run(port=free_port, host="0.0.0.0")
threading.Thread(target=run_flask, daemon=True).start()
time.sleep(3)

# === INSTALL & RUN CLOUDFLARED ===
if shutil.which('cloudflared') is None:
    print('Installing cloudflared...')
    subprocess.run(['wget', 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb'])
    subprocess.run(['dpkg', '-i', 'cloudflared-linux-amd64.deb'])

print(f"Starting Cloudflared tunnel to expose port {free_port}...")

cloudflared_proc = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', f'http://localhost:{free_port}'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

public_url = None
for line in cloudflared_proc.stdout:
    print(line, end='')  # Show logs for debugging
    match = re.search(r'(https://[a-zA-Z0-9\-]+\.trycloudflare\.com)', line)
    if match:
        public_url = match.group(1)
        break

if public_url:
    print(f"\nYour public endpoint is: {public_url}/generate")
else:
    print("Failed to get public URL from cloudflared. Check logs.")


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8006
 * Running on http://172.28.0.12:8006
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an a

Starting Cloudflared tunnel to expose port 8006...
2025-07-15T17:59:48Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-07-15T17:59:48Z INF Requesting new quick Tunnel on trycloudflare.com...


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

2025-07-15T17:59:53Z INF +--------------------------------------------------------------------------------------------+
2025-07-15T17:59:53Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-07-15T17:59:53Z INF |  https://kirk-referred-pendant-builder.trycloudflare.com                                   |

Your public endpoint is: https://kirk-referred-pendant-builder.trycloudflare.com/generate


In [ ]:

# --- Kill any old servers/tunnels for a clean Colab state
import os, time
os.system("pkill -f flask")
os.system("pkill -f cloudflared")
time.sleep(2)

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from flask import Flask, request, jsonify
import threading, socket, subprocess, re, shutil, time

# --- You MUST define your model path in advance (no reassign here)
# For example:
# MODEL_PATH = "./"  # Or wherever your model files are

# --- Find free port
def find_free_port(start=8001, end=8100):
    for port in range(start, end):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('127.0.0.1', port)) != 0:
                return port
    raise RuntimeError("No free port found!")

# --- Load your model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# --- Flask app - Using MANUAL CORS headers (no flask_cors)
app = Flask(__name__)

# NOT using flask_cors to avoid duplicate headers

def generate_text(prompt, max_length=250, num_return_sequences=1, temperature=0.8):
    try:
        inputs = tokenizer(prompt, return_tensors="pt", padding=True)
        input_ids = inputs.input_ids.to(device)
        attention_mask = inputs.attention_mask.to(device)
        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=len(input_ids[0]) + max_length,
                do_sample=True,
                temperature=temperature,
                top_k=50,
                top_p=0.95,
                num_return_sequences=num_return_sequences,
                pad_token_id=tokenizer.pad_token_id if tokenizer.pad_token_id else tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        generated_texts = []
        for output in outputs:
            generated = tokenizer.decode(output[input_ids.shape[1]:], skip_special_tokens=True)
            generated_texts.append(generated.strip())
        return generated_texts
    except Exception as e:
        return [f"Error: {str(e)}"]

@app.route('/generate', methods=['POST', 'OPTIONS'])
def generate():
    # Handle preflight OPTIONS request
    if request.method == 'OPTIONS':
        response = jsonify({})
        response.headers.add('Access-Control-Allow-Origin', '*')
        response.headers.add('Access-Control-Allow-Headers', 'Content-Type,Authorization')
        response.headers.add('Access-Control-Allow-Methods', 'POST,OPTIONS')
        return response

    # Handle actual POST request
    try:
        data = request.get_json()
        if not data:
            return jsonify({"error": "No JSON data provided"}), 400

        prompt = data.get("prompt", "")
        if not prompt:
            return jsonify({"error": "No prompt provided"}), 400

        max_length = int(data.get("max_length", 250))
        num_return_sequences = int(data.get("num_return_sequences", 1))
        temperature = float(data.get("temperature", 0.8))

        results = generate_text(
            prompt,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            temperature=temperature
        )

        response = jsonify({"generations": results})
        response.headers.add('Access-Control-Allow-Origin', '*')
        return response

    except Exception as e:
        error_response = jsonify({"error": str(e)})
        error_response.headers.add('Access-Control-Allow-Origin', '*')
        return error_response, 500

# Add a health check endpoint
@app.route('/health', methods=['GET'])
def health():
    response = jsonify({"status": "healthy", "device": device})
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response

# --- Run Flask in a thread
free_port = find_free_port()
def run_flask():
    app.run(port=free_port, host="0.0.0.0", debug=False)

threading.Thread(target=run_flask, daemon=True).start()
time.sleep(3)

# --- Install and run cloudflared
if shutil.which('cloudflared') is None:
    print('Installing cloudflared...')
    subprocess.run(['wget', 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb'])
    subprocess.run(['dpkg', '-i', 'cloudflared-linux-amd64.deb'])

print(f"Starting Cloudflared tunnel to expose port {free_port}...")

cloudflared_proc = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', f'http://localhost:{free_port}'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

public_url = None
for line in cloudflared_proc.stdout:
    print(line, end='')  # Show logs for debugging
    match = re.search(r'(https://[a-zA-Z0-9\-]+\.trycloudflare\.com)', line)
    if match:
        public_url = match.group(1)
        break

if public_url:
    print(f"\nYour public endpoint is: {public_url}/generate")
    print(f"Health check endpoint: {public_url}/health")
    print(f"\nExample usage:")
    print(f"curl -X POST {public_url}/generate \\")
    print(f"  -H 'Content-Type: application/json' \\")
    print(f"  -d '{{\"prompt\": \"Hello world\", \"max_length\": 100}}'")
else:
    print("Failed to get public URL from cloudflared. Check logs.")

# Keep the script running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\nShutting down...")
    cloudflared_proc.terminate()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8012
 * Running on http://172.28.0.12:8012
INFO:werkzeug:Press CTRL+C to quit


Starting Cloudflared tunnel to expose port 8012...
2025-07-15T18:12:53Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-07-15T18:12:53Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-07-15T18:12:57Z INF +--------------------------------------------------------------------------------------------+
2025-07-15T18:12:57Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-07-15T18:12:57Z INF |  ht

INFO:werkzeug:127.0.0.1 - - [15/Jul/2025 18:13:21] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jul/2025 18:14:45] "OPTIONS /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jul/2025 18:14:52] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jul/2025 18:15:28] "OPTIONS /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jul/2025 18:15:40] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jul/2025 18:15:55] "OPTIONS /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jul/2025 18:16:08] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jul/2025 18:44:18] "GET /generate HTTP/1.1" 405 -
